# PNAS paper submission analysis

Analysing the wind characteristics of streaked shearwaters as they approach foraging points allows us to examine what conditions the birds travel in throughout foraging trips. 

## Previous study

In the most similar previous study ([Nevitt et al., 2008](https://www.pnas.org/content/105/12/4576)) investigating similar phenomena, the tracks of albatross approaching foraging points were examined and classified by their characteristics (in 4 groups: Direct, Turn, Zigzag, and Circle):

<p align="center">
   <img src=https://www.pnas.org/cms/10.1073/pnas.0709047105/asset/eb810a4f-38f4-4a23-beee-eb585774c043/assets/graphic/zpq0080896790001.jpeg alt="Albatross tracks approaching foraging" width="400">
</p>

And the histograms of relative wind bearings were examined for those categories:

 <p align="center">
    <img src=https://www.pnas.org/cms/10.1073/pnas.0709047105/asset/7d4e5b13-0f76-4fc1-b230-f77569c6a300/assets/graphic/zpq0080896790002.jpeg alt="Relative wind histograms" width="400">
 </p>
<!-- 
<p align="center>
   <img src= alt="Relative wind histograms">
</p> -->

This study used recordings of wandering albatross with GPS and stomach temperature transmitters. Foraging points were estimates as periods with rapid drops in stomach temperature, and surface landings taken from times when birds moved at under 2.8 m/s. Wind directions and speeds from all touchdown points were estimated using QuikSCAT daily level 3 gridded ocean wind vectors. This data comes from the SeaWinds scatterometer and records wind vector fields twice daily. Wind values cannot be produced for locations within 30km of land/ice. Backscatter recorded by the satellite is then processed through a Ku-band geophysical model function which derives surface wind speeds and direction. Rain contamination can be an issue due to difficulty in separating the backscatter effect of wind vs rain. The level value refers to the processing level of the data, starting at 0 (raw) through to 3 or 4.

In [7]:
using DataFrames, CSV, RCall, Plots, Geodesy, Dates, Distances, Statistics, Glob

In [3]:
# read in the foraging estimates and GPS tracks
if Sys.iswindows()
    fileloc = "E:/My Drive/PhD/Data/2018Shearwater/TxtDat/AxyTrek/BehaviourDetection/PredictedForage/"
else
    fileloc = "/Volumes/GoogleDrive/My Drive/PhD/Data/2018Shearwater/TxtDat/AxyTrek/BehaviourDetection/PredictedForage/"
end
EFiles = glob("*ForageGPS.txt", fileloc)
EDat = DataFrame()
# for b = EFiles
#     append!(EDat, CSV.read(b, DataFrame), cols = :union)
# end

""
""


In [25]:
EFiles[3]
match(r"*PredictedForage/(d_))

ErrorException: syntax: incomplete: invalid string syntax

In [ ]:


# load in the wind calculations
if Sys.iswindows()
    estLoc = "E:/My Drive/PhD/Data/2018Shearwater/WindEst/MinDat/"
else
    estLoc = "/Volumes/GoogleDrive/My Drive/PhD/Data/2018Shearwater/WindEst/MinDat/"
end
# estLoc = "/Volumes/GoogleDrive/My Drive/PhD/Data/WindEstTest/2018/"
cd(estLoc)
EFiles = readdir(estLoc)
EDat = []
for b = 1:length(EFiles)
    if b == 1
        EDat = CSV.File(string(estLoc,EFiles[b]), header = false) |> DataFrame
        EDat.ID = fill("18"*EFiles[b][1:end-4],nrow(EDat))
    else Add = CSV.File(string(estLoc,EFiles[b]), header = false) |> DataFrame
        Add.ID = fill("18"*EFiles[b][1:end-4],nrow(Add))
        EDat = vcat(EDat, Add)
    end
end
# repeat for 2019
if Sys.iswindows()
    estLoc = "E:/My Drive/PhD/Data/2019Shearwater/WindEst/MinDat/"
else
    estLoc = "/Volumes/GoogleDrive/My Drive/PhD/Data/2019Shearwater/WindEst/MinDat/"
end
# estLoc = "/Volumes/GoogleDrive/My Drive/PhD/Data/WindEstTest/2018/"
cd(estLoc)
EFiles = readdir(estLoc)
EFiles = EFiles[contains.(EFiles,".csv")]
EDat19 = []
for b = 1:length(EFiles)
    if b == 1
        EDat19 = CSV.File(string(estLoc,EFiles[b]), header = false) |> DataFrame
        EDat19.ID = fill("19"*EFiles[b][1:end-4],nrow(EDat19))
    else Add19 = CSV.File(string(estLoc,EFiles[b]), header = false) |> DataFrame
        Add19.ID = fill("19"*EFiles[b][1:end-4],nrow(Add19))
        EDat19 = vcat(EDat19, Add19)
    end
end
EDat = vcat(EDat,EDat19)